# Segmenting and Clustering Neighborhoods in Toronto

In [50]:
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import requests
import ssl
import re
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


import json # library to handle JSON files


from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium

import folium

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,

In [51]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(url, 'lxml')

In [52]:
x = soup.find('table', {'class':'wikitable sortable'})
print(x)

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [53]:
column_names = ['Postcode','Borough','Neighborhood']
tor = pd.DataFrame(columns = column_names)
postcode = 0
borough = 0
neighborhood = 0

In [54]:
for tr in x.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text[0:-1]
    tor = tor.append({'Postcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
    print(postcode, borough, neighborhood)



0 0 0
M1A Not assigned Not assigned
M2A Not assigned Not assigned
M3A North York Parkwoods
M4A North York Victoria Village
M5A Downtown Toronto Harbourfront
M5A Downtown Toronto Regent Park
M6A North York Lawrence Heights
M6A North York Lawrence Manor
M7A Queen's Park Not assigned
M8A Not assigned Not assigned
M9A Etobicoke Islington Avenue
M1B Scarborough Rouge
M1B Scarborough Malvern
M2B Not assigned Not assigned
M3B North York Don Mills North
M4B East York Woodbine Gardens
M4B East York Parkview Hill
M5B Downtown Toronto Ryerson
M5B Downtown Toronto Garden District
M6B North York Glencairn
M7B Not assigned Not assigned
M8B Not assigned Not assigned
M9B Etobicoke Cloverdale
M9B Etobicoke Islington
M9B Etobicoke Martin Grove
M9B Etobicoke Princess Gardens
M9B Etobicoke West Deane Park
M1C Scarborough Highland Creek
M1C Scarborough Rouge Hill
M1C Scarborough Port Union
M2C Not assigned Not assigned
M3C North York Flemingdon Park
M3C North York Don Mills South
M4C East York Woodbine Hei

In [55]:
tor = tor[tor.Borough!='Not assigned']
tor = tor[tor.Neighborhood!='Not assigned']
tor = tor[tor.Borough!= 0]
tor.reset_index(drop = True, inplace = True)

tor = tor.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

tor

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [56]:
tor.shape

(102, 3)

In [57]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_dff8a113d75e412297d555c23139b4ae = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='A1hpE2gUdK0I7nXY9z-EkAGvnh7zYopG7cZcYvRI-mxl',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_dff8a113d75e412297d555c23139b4ae.get_object(Bucket='myjupyternotebookonibmdatascience-donotdelete-pr-sdqy5hblun8tew',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.rename(columns={'Postal Code':'Postcode'}, inplace=True)

df_data_1.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [58]:
tor.set_index('Postcode')
df_data_1.set_index('Postcode')
toronto_data = pd.merge(tor, df_data_1)
toronto_data

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [59]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


In [60]:
# create map of New York using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [61]:
CLIENT_ID = 'KFCMPSJJJKYAMYNH0DTGZ3AMLNOIQ5S5E0ZN1QI2J5PZNLNT' # your Foursquare ID
CLIENT_SECRET = 'CZ2KFRPJSMDIDXLM5EAXENRTKR33SVG2ZPCJTZQZVCD5DMRJ' # your Foursquare Secret
VERSION = '20190501' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KFCMPSJJJKYAMYNH0DTGZ3AMLNOIQ5S5E0ZN1QI2J5PZNLNT
CLIENT_SECRET:CZ2KFRPJSMDIDXLM5EAXENRTKR33SVG2ZPCJTZQZVCD5DMRJ


In [62]:
toronto_data.loc[67, 'Neighborhood']

'Chinatown, Grange Park, Kensington Market'

In [63]:
neighborhood_latitude = toronto_data.loc[67, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[67, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[67, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Chinatown, Grange Park, Kensington Market are 43.6532057, -79.4000493.


In [64]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KFCMPSJJJKYAMYNH0DTGZ3AMLNOIQ5S5E0ZN1QI2J5PZNLNT&client_secret=CZ2KFRPJSMDIDXLM5EAXENRTKR33SVG2ZPCJTZQZVCD5DMRJ&v=20190501&ll=43.6532057,-79.4000493&radius=500&limit=100'

In [65]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cca6f8fdd57972411e89c88'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-50427a03e4b08d9f5931f593-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1c1941735',
         'name': 'Mexican Restaurant',
         'pluralName': 'Mexican Restaurants',
         'primary': True,
         'shortName': 'Mexican'}],
       'id': '50427a03e4b08d9f5931f593',
       'location': {'address': '69 Kensington Ave',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Baldwin St',
        'distance': 140,
        'formattedAddress': ['69 Kensington Ave (Baldwin St)',
         'Toronto ON M5T 2L6',
         'Canada'],
        'labeled

In [66]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [67]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Seven Lives - Tacos y Mariscos,Mexican Restaurant,43.654418,-79.400545
1,Kid Icarus,Arts & Crafts Store,43.653933,-79.401719
2,Essence of Life Organics,Organic Grocery,43.654111,-79.400431
3,The Moonbean Cafe,Café,43.654147,-79.400182
4,Little Pebbles,Coffee Shop,43.654883,-79.400264
5,Jimmy's Coffee,Café,43.654493,-79.401311
6,FIKA Cafe,Café,43.653560,-79.400402
7,Banh Mi Nguyen Huong,Vietnamese Restaurant,43.653628,-79.398376
8,Cold Tea,Bar,43.654193,-79.401075
9,El Rey,Cocktail Bar,43.652764,-79.400048


In [68]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.
